# Temporal Face Feature
## Activation of AUs over time

In [4]:
import numpy as np
import pandas as pd
import logging

In [5]:
# Step 1: Load landmarks data
df_happy = pd.read_csv("./AU_data/happy_fed.csv")
df_neutral = pd.read_csv("./AU_data/neutral_fed.csv")

In [6]:
# remove leading whitespaces in column names
def remove_whitespaces(df):
    df.columns = [col.strip() for col in df.columns]

remove_whitespaces(df_happy)
remove_whitespaces(df_neutral)

In [7]:
def get_required_cols(df):
    au = df.filter(like='AU')
    tmp = df[['frame', 'timestamp']]
    return pd.concat([tmp, au], axis=1)

df_happy = get_required_cols(df_happy)
df_neutral = get_required_cols(df_neutral)

In [8]:
df_happy

,frame,timestamp,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0.464,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.801,0.10,0.05,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.856,0.11,0.06,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.903,0.05,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.934,0.02,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,116,5.025,0.61,0.06,0.0,0.0,0.07,1.14,0.0,0.41,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,117,5.063,0.58,0.13,0.0,0.0,0.00,1.09,0.0,0.35,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117,118,5.098,0.58,0.13,0.0,0.0,0.00,1.10,0.0,0.30,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
118,119,5.145,0.54,0.10,0.0,0.0,0.00,0.97,0.0,0.21,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### AUs for Smile
Relevant AUs for smiling: AU6 (cheek raiser) and AU12 (lip corner puller) (src gathered [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7785114/#:~:text=A%20smile%20is%20mainly%20concerned,very%20often%20involved%20as%20well.) or at Face_3 slides s.102 where they also say AU25 (Lips part))


We are using OpenFace 2.0, which is based on Dlib and uses 68 coordinates that map to facial structures of the face (https://pyimagesearch.com/2017/04/03/facial-landmarks-dlib-opencv-python/)



Note: here a paper, where they point out that AU6 and AU12 were lower for spontaneous smiles than posed smiles (https://www.mdpi.com/1424-8220/20/4/1199) &rightarrow; in our case, a child has to do a posed smile to reload the robot, so we are good with our way to go.

Note: `_r` is for intensity, and `_c` for activation.

In [9]:
#TODO: should we include logic for when too many other AUs are activated? maybe that the user needs to try harder?
happy_AUs = df_happy[["frame", "timestamp", "AU06_r","AU12_r", "AU25_r", "AU06_c","AU12_c", "AU25_c"]]
happy_AUs

,frame,timestamp,AU06_r,AU12_r,AU25_r,AU06_c,AU12_c,AU25_c
0,1,0.464,0.00,0.00,0.00,0.0,0.0,0.0
1,2,0.801,0.00,0.00,0.00,0.0,0.0,0.0
2,3,0.856,0.00,0.00,0.05,0.0,0.0,0.0
3,4,0.903,0.00,0.00,0.05,0.0,0.0,0.0
4,5,0.934,0.00,0.00,0.05,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
115,116,5.025,0.07,0.69,1.43,0.0,1.0,0.0
116,117,5.063,0.00,0.44,0.72,0.0,1.0,0.0
117,118,5.098,0.00,0.28,0.59,0.0,1.0,0.0
118,119,5.145,0.00,0.22,0.53,0.0,1.0,0.0


In [10]:
#TODO: check ffps rate of Openface we used, either 25 or 30
#TODO: check if activation is one that the intensity only then has a value

Calculate means of activations to determine if the player smiled or not.

For now, we will use a threshold of activation = 0.6.

All three AUs need to pass that value so that the window_size can be counted as a "smile window" (for now, window_size is set to 30)

TODO: check if correct like that

In [48]:
WINDOW_SIZE = 30

#TODO: check for sliding window within this time frame so that there are overlaps

def check_exceed_threshold(mean_AU_value_list, threshold=0.6):
    return all(mean_val >= threshold for mean_val in mean_AU_value_list)

def mean_activation_per_window(data, num_seconds=2, step_size=15):
    """
    Parameters:
        - data: DataFrame
            the required columns, like AU activations and frames, needed for further processing
        - num_seconds: int
            number of seconds required to "hold" the emotion
    """
    # extract start and end frame number
    start_frame = data['frame'].iloc[0]
    end_frame = data['frame'].iloc[-1] + 1
    current_frame = start_frame

    num_exceeded = 0
    while current_frame < end_frame:
        print("------------------")
        if num_exceeded >= num_seconds:
            break
        remaining_frames = end_frame - current_frame
        # drop the last frames as they cannot be taken into account as a "whole" second
        if remaining_frames < WINDOW_SIZE:
            logging.info(f"We will drop the remaining {remaining_frames} frames.")
            break
        window_start = current_frame
        # Ensure not to exceed end_frame
        window_end = min(current_frame + WINDOW_SIZE + 1, end_frame)
        window_data = data[(data['frame'] >= window_start) & (data['frame'] < window_end)]
        print(window_data["frame"].iloc[0], window_data["frame"].iloc[-1], len(window_data))
        mean_AU_value_list = list(window_data[["AU06_c", "AU12_c", "AU25_c"]].mean())
        print(mean_AU_value_list)

        #TODO: use logging instead of print!
        if check_exceed_threshold(mean_AU_value_list):
            num_exceeded += 1
            print(f"Smile is detected for n={num_exceeded}.")
        else:
            # reset window size of smiling
            num_exceeded = 0
            print("No smile detected")

        current_frame += step_size


    if num_exceeded >= num_seconds:
        print("Smile was held long enough. B will be reloaded now.")
    elif (num_exceeded < num_seconds) and (num_exceeded > 0):
        print("Smile was held too short to reload B completely. It has to be tried again.")
    else:
        print("Smile task was not executed like required. Display details in the UI.")


mean_activation_per_window(happy_AUs[:-43], num_seconds=2, step_size=15)

------------------
1 31 31
[0.0, 0.0, 0.0]
No smile detected
------------------
16 46 31
[0.0, 0.0, 0.0]
No smile detected
------------------
31 61 31
[0.3548387096774194, 0.45161290322580644, 0.22580645161290322]
No smile detected
------------------
46 76 31
[0.8387096774193549, 0.9354838709677419, 0.7096774193548387]
Smile is detected for n=1.
------------------
Smile was held too short to reload B completely. It has to be tried again.
